In [1]:
!pip3 install thirdai
!pip3 install pandas # Needed by the datasets module to convert datasets to CSV format
!pip3 install numpy

import datasets

train_filename, test_filename, inference_batch = datasets.download_clinc()

In [2]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "text": bolt.types.text(),
        "category": bolt.types.categorical(n_unique_classes=151)
    },
    target="category"
)


======================= Bolt Model =======================
input_1 (Input): dim=200000
input_1 -> fc_1 (FullyConnected): dim=512, sparsity=1, act_func=ReLU
fc_1 -> fc_2 (FullyConnected): dim=151, sparsity=1, act_func=Softmax



In [3]:
train_config = (bolt.TrainConfig(epochs=5, learning_rate=0.01)
                    .with_metrics(["categorical_accuracy"]))

model.train(train_filename, train_config)

test_config = (bolt.EvalConfig()
                   .with_metrics(["categorical_accuracy"]))

model.evaluate(test_filename, test_config)

Loading vectors from './clinc_train.csv'
Loaded 15000 vectors from './clinc_train.csv' in 0 seconds.
train epoch 0:

[==================================================] 100%

train | epoch 0 | updates 8 | {categorical_accuracy: 0.3792} | batches 8 | time 0s | complete

train epoch 1:

[==================================================] 100%

train | epoch 1 | updates 16 | {categorical_accuracy: 0.857133} | batches 8 | time 0s | complete

train epoch 2:

[==================================================] 100%

train | epoch 2 | updates 24 | {categorical_accuracy: 0.930467} | batches 8 | time 0s | complete

train epoch 3:

[==================================================] 100%

train | epoch 3 | updates 32 | {categorical_accuracy: 0.968533} | batches 8 | time 0s | complete

train epoch 4:

[==================================================] 100%===========================      ] 87%

train | epoch 4 | updates 40 | {categorical_accuracy: 0.987133} | batches 8 | time 0s | complete


array([[9.9385816e-01, 4.7021664e-11, 3.1668950e-08, ..., 2.4105037e-10,
        3.3457324e-11, 1.1335135e-19],
       [9.0282941e-01, 3.7333148e-10, 8.0230443e-08, ..., 4.7793971e-08,
        2.8450900e-10, 5.4974688e-18],
       [9.9930882e-01, 6.6613226e-10, 2.5294650e-08, ..., 1.7064009e-08,
        5.9939049e-09, 1.3808543e-17],
       ...,
       [2.2775976e-11, 5.3681096e-12, 6.4077227e-13, ..., 5.9045919e-09,
        9.9916208e-01, 4.8098624e-22],
       [1.3998407e-06, 1.3461320e-09, 1.7499859e-10, ..., 1.6464426e-07,
        2.1315204e-01, 2.3931525e-17],
       [2.0083364e-08, 8.9741592e-10, 1.2640340e-10, ..., 3.6856754e-08,
        9.9675447e-01, 6.1367484e-17]], dtype=float32)

In [4]:
print(inference_batch)

[{'text': 'can you please do a simulated coin flip for me ai'}, {'text': 'would you call yourself a human or a computer'}, {'text': 'would it be ok to use butter instead of oil'}, {'text': 'how much do i have available on my visa card'}, {'text': 'how long does pizza take'}]


In [5]:
import numpy as np

prediction = model.predict(inference_batch[0])
prediction_batch = model.predict_batch(inference_batch)

print(model.class_name(np.argmax(prediction)))

flip_coin
